In [1]:
%matplotlib inline
from Prediction import*

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
from bokeh.layouts import column
output_file('Test_Preiction.html')



In [2]:
data = pd.read_csv('../Data/DrahiX_Data.csv', index_col=0, parse_dates=True)
data = data.resample('15Min').mean()
data = data.fillna(method='ffill')

#Adding new features so that we can use the date and time in the model
data['day of the week'] = data.index.dayofweek
data['day of the year'] = data.index.dayofyear
data['hour of the day'] = data.index.hour
data['minute of the hour'] = data.index.minute
data["TGBT"] = data['T1']+data['T2']+data['T3']+data['T4']


In [3]:
def predict(data, size):

    tf.compat.v1.reset_default_graph()

    sequence_length = int(size)  

    #Organizing the data and keeping only the columns that will be used in the model
    features = ['day of the week','day of the year','hour of the day','minute of the hour', 'AirTemp','rh']
    labels   = ['TGBT']
    inputs   = features + labels
    data = data[inputs]

    num_features = len(features)
    num_labels = len(labels)
    num_inputs = num_features + num_labels

    #Normalizing data between 1 and -1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    
    data_scaled = pd.DataFrame(scaler.fit_transform(data.values), columns=data.columns, index=data.index)
    x_test_scaled = np.asarray(data_scaled[features])
    y_test_scaled = np.asarray(data_scaled[labels])


    sess=tf.Session()   
    #First let's load meta graph and restore weights
    saver = tf.train.import_meta_graph('Model'+str(size)+'/my_test_model-1000.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./Model'+str(size)+'/'))
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    outputs = graph.get_tensor_by_name("op_to_restore:0")

    y_pred = sess.run(outputs, feed_dict={x: [x_test_scaled[:size]]})
    y_test = y_pred.reshape(-1, num_labels)
    scaler.fit(data[labels].iloc[:size])

    DF = pd.DataFrame(scaler.inverse_transform(y_test), index=data.iloc[:size].iloc[0:sequence_length].index + pd.Timedelta(minutes=15*sequence_length), columns=labels)
    
    
    for i in range(int(len(data)/size - 2)):
            scaler = MinMaxScaler(feature_range=(-1, 1))
            data_scaled = pd.DataFrame(scaler.fit_transform(data.iloc[size + size*i:2*size+size*i].values), columns=data.iloc[size + size*i:2*size+size*i].columns, index=data.iloc[size + size*i:2*size+size*i].index)
            x_test_scaled = np.asarray(data_scaled[features])
            y_test_scaled = np.asarray(data_scaled[labels])
        
            y_pred = sess.run(outputs, feed_dict={x: [x_test_scaled]})
            y_test = y_pred.reshape(-1, num_labels)
            scaler.fit(data[labels].iloc[size + size*i:2*size+size*i])
    
            result = pd.DataFrame(scaler.inverse_transform(y_test), index=data.iloc[size + size*i:2*size+size*i].iloc[0:sequence_length].index + pd.Timedelta(minutes=15*sequence_length), columns=labels)
            DF = pd.concat([DF,result])
        
    return DF

In [5]:
result12 = predict(data, 12)
result24 = predict(data, 24)
result96 = predict(data, 96)   

INFO:tensorflow:Restoring parameters from ./Model12/my_test_model-1000
INFO:tensorflow:Restoring parameters from ./Model24/my_test_model-1000
INFO:tensorflow:Restoring parameters from ./Model96/my_test_model-1000


In [6]:

result12["TGBT"] = (result12["TGBT"] - data["TGBT"].min())/(data["TGBT"].max() - data["TGBT"].min())
result24["TGBT"] = (result24["TGBT"] - data["TGBT"].min())/(data["TGBT"].max() - data["TGBT"].min())
result96["TGBT"] = (result96["TGBT"] - data["TGBT"].min())/(data["TGBT"].max() - data["TGBT"].min())
data["TGBT"] = (data["TGBT"] - data["TGBT"].min())/(data["TGBT"].max() - data["TGBT"].min())


In [7]:
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error


mse12 = mean_squared_error(result12["TGBT"], data["TGBT"].iloc[12:-1])
rmse12 = sqrt(mse12)
mae12 = mean_absolute_error(result12["TGBT"], data["TGBT"].iloc[12:-1])

mse24 = mean_squared_error(result24["TGBT"], data["TGBT"].iloc[24:-1])
rmse24 = sqrt(mse24)
mae24 = mean_absolute_error(result24["TGBT"], data["TGBT"].iloc[24:-1])

mse96 = mean_squared_error(result96["TGBT"], data["TGBT"].iloc[96:-1])
rmse96 = sqrt(mse96)
mae96 = mean_absolute_error(result96["TGBT"], data["TGBT"].iloc[96:-1])


print('MSE : %.3f' % mse12)
print('MAE : %.3f' % mae12)
print('RMSE : %.3f' % rmse12)

print('\nMSE : %.3f' % mse24)
print('MAE : %.3f' % mae24)
print('RMSE : %.3f' % rmse24)

print('\nMSE : %.3f' % mse96)
print('MAE : %.3f' % mae96)
print('RMSE : %.3f' % rmse96)

MSE : 0.006
MAE : 0.050
RMSE : 0.076

MSE : 0.009
MAE : 0.068
RMSE : 0.096

MSE : 0.009
MAE : 0.069
RMSE : 0.095


In [8]:

p1 = figure(x_axis_type="datetime",plot_width=1200)
p1.title.text ='Model 3h'+'\t\t\tMSE : %.3f' % mse12 + '\t\t\tMAE : %.3f' % mae12 + '\t\t\tRMSE : %.3f' % rmse12
p1.line(x='Date and time (UTC)', y='TGBT', line_width=2, source=data, legend='Real Consumption')
p1.line(x='Date and time (UTC)', y='TGBT', line_width=2, source=result12, color=Spectral3[2],legend='Prediction', alpha = 0.9)

p2 = figure(x_axis_type="datetime",plot_width=1200)
p2.title.text ='Model 6h'+'\t\t\tMSE : %.3f' % mse24 + '\t\t\tMAE : %.3f' % mae24 + '\t\t\tRMSE : %.3f' % rmse24
p2.line(x='Date and time (UTC)', y='TGBT', line_width=2, source=data, legend='Real Consumption')
p2.line(x='Date and time (UTC)', y='TGBT', line_width=2, source=result24, color=Spectral3[2],legend='Prediction',alpha = 0.9)

p3 = figure(x_axis_type="datetime",plot_width=1200)
p3.title.text ='Model 24h'+'\t\t\tMSE : %.3f' % mse96 + '\t\t\tMAE : %.3f' % mae96 + '\t\t\tRMSE : %.3f' % rmse96
p3.line(x='Date and time (UTC)', y='TGBT', line_width=2, source=data, legend='Real Consumption')
p3.line(x='Date and time (UTC)', y='TGBT', line_width=2, source=result96, color=Spectral3[2],legend='Prediction',alpha = 0.9)


show(column(p1,p2,p3))